In [108]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pickle
import random
from copy import copy, deepcopy
from scipy.io import arff
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.spatial.distance import pdist, cdist
from sklearn.metrics.pairwise import euclidean_distances, pairwise_distances
from sklearn.utils import check_random_state
from scipy.spatial import distance
from tslearn.soft_dtw_fast import *
from tslearn.cysax import *
from tslearn.metrics import dtw, dtw_path,dtw_path_from_metric,soft_dtw
from tslearn.utils import *


# Loading Training and Testing Data Source

In [109]:
data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TRAIN.arff')
data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TRAIN.arff')
data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TRAIN.arff')

Test_data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TEST.arff')
Test_data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TEST.arff')
Test_data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TEST.arff')

In [110]:
df_D1 = pd.DataFrame(data1[0])
df_D2 = pd.DataFrame(data2[0])
df_D3 = pd.DataFrame(data3[0])

Test_df_D1 = pd.DataFrame(Test_data1[0])
Test_df_D2 = pd.DataFrame(Test_data2[0])
Test_df_D3 = pd.DataFrame(Test_data3[0])

## Make Over the data, as desired

In [111]:
df_D1 =df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
df_D2 =df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
df_D3 =df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

Test_df_D1 =Test_df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
Test_df_D2 =Test_df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
Test_df_D3 =Test_df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

In [112]:
df_D1 = df_D1.drop('target', axis=1)
df_D2 = df_D2.drop('target', axis=1)
df_D3 = df_D3.drop('target', axis=1)

Test_df_D1 = Test_df_D1.drop('target', axis=1)
Test_df_D2 = Test_df_D2.drop('target', axis=1)
Test_df_D3 = Test_df_D3.drop('target', axis=1)

In [113]:
Sort_Dim_1 = np.expand_dims(df_D1.values.reshape(-1),axis=1)
Sort_Dim_2 = np.expand_dims(df_D2.values.reshape(-1),axis=1)
Sort_Dim_3 = np.expand_dims(df_D3.values.reshape(-1),axis=1)

Sort_Test_Dim_1 = np.expand_dims(Test_df_D1.values.reshape(-1),axis=1)
Sort_Test_Dim_2 = np.expand_dims(Test_df_D2.values.reshape(-1),axis=1)
Sort_Test_Dim_3 = np.expand_dims(Test_df_D3.values.reshape(-1),axis=1)

In [114]:
Training_View = np.concatenate((Sort_Dim_1, Sort_Dim_2),axis=1)
Training_View = np.concatenate((Training_View, Sort_Dim_3),axis=1)

In [115]:
Training_View.shape

(258804, 3)

In [116]:
Testing_View = np.concatenate((Sort_Test_Dim_1, Sort_Test_Dim_2),axis=1)
Testing_View = np.concatenate((Testing_View, Sort_Test_Dim_3),axis=1)

In [117]:
finalarray_test_data = Testing_View[:][0:182]

In [118]:
finalarray_test_data.shape

(182, 3)

## Initializing the data

In [119]:
Total_rows = 261352
Sample_Percentage = 0
Number_of_run= 0
Length_RTS = 0
global Sub_Sample_copy_PTS 
Training_Results_List = []
PTS_Results_List = []
Store_Avg_list = []
dictionary = dict()
dictionary_RTS = dict()
dictionary_RTS_all = dict()
dictionary_PTS = dict()
dictionary_PTS_for_all_change = dict()
RTS_short_list_Results_List = []
Starting_point = Testing_View[:][0:1]
End_point = Testing_View[:][181:182]

## Functions body for Random TS

In [120]:
def Calculate_Percentage(Sample_Percentage):
    Refining_Percentage= Total_rows * (Sample_Percentage/100)
    Refining_Percentage = round(Refining_Percentage)   
    return Refining_Percentage

In [121]:
def Get_equivalent_rows(Refining_Percentage):
    Sub_Sampling = Training_View[:][0:Refining_Percentage]
    return Sub_Sampling    

In [122]:
def Get_random_row(Sub_Sampling,Length_RTS):
    
    number_of_rows = Sub_Sampling.shape[0]
    random_indices = np.random.choice(number_of_rows, size=Length_RTS, replace=False)
    random_rows = Sub_Sampling[random_indices, :]
    return random_rows
    

In [123]:
def Adding_start_end_point(random_rows):
    Output_TS = np.concatenate((Starting_point, random_rows),axis=0)
    Output_TS = np.concatenate((Output_TS, End_point),axis=0)
    return Output_TS

In [124]:
def Calculate_Random_DTW(Output_TS):
    path, dist = dtw_path(finalarray_test_data, Output_TS)
    if(dist>0):
        Training_Results_List.append(dist)  


# CODE 

In [106]:
# for y in range(1,10):
    
#     Refining_Percentage = Calculate_Percentage(Given_Percentage)
#     Sub_Sampling = Get_equivalent_rows(Refining_Percentage)
#     Iteration_number = len(Sub_Sampling) /182
#     Iteration_number_t = round (Iteration_number)

#     p = 181
    
#     for i in range (1,Iteration_number_t):

#         random_rows = Get_random_row(Sub_Sampling,random_num_out)
#         Output_TS = Adding_start_end_point(random_rows)
#         Calculate_Random_DTW(Output_TS) 
        
#     dictionary_RTS_all[y] = Training_Results_List
#     Training_Results_List= []

In [133]:
Given_Percentage = 20
random_num_out = 60

In [135]:
for y in range(1,3):
    
    Refining_Percentage = Calculate_Percentage(Given_Percentage)
    Sub_Sampling = Get_equivalent_rows(Refining_Percentage)
    Iteration_number = len(Sub_Sampling) /182
    Iteration_number_t = round (Iteration_number)

    p = 181
    
    for i in range (1,Iteration_number_t):

        random_rows = Get_random_row(Sub_Sampling,random_num_out)
        Output_TS = Adding_start_end_point(random_rows)
        Calculate_Random_DTW(Output_TS) 
        
        if(y==1):

            Sub_Sampling_PTS = Sub_Sampling[:][1:181*y]
            Pseudo_TS = Sub_Sampling_PTS
            Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS,y)
            Run_all_methods(Complete_PTS) 

        else:

            Sub_Sampling_PTS = Sub_Sampling[:][1+p:181*y]
            Pseudo_TS = Sub_Sampling_PTS
            Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS,y)
            Run_all_methods(Complete_PTS) 
        
        
        
        
    dictionary_RTS_all[y] = Training_Results_List
    Training_Results_List= []
    dictionary_PTS_for_all_change[y] = PTS_Results_List
    PTS_Results_List = []

In [136]:
dictionary_RTS_all

{1: [15.132475423222237,
  15.573367470027542,
  15.128825659343885,
  14.171640953486367,
  13.6650993491877,
  13.934860050613748,
  15.071031628837094,
  15.310048230111619,
  13.600069153260913,
  13.255587259170943,
  13.38084714808853,
  14.752048089145216,
  14.07989324918115,
  14.595944376287646,
  14.01616704183344,
  15.560187725263955,
  15.79320388692418,
  13.921137771134692,
  13.01832911694193,
  15.770897120265547,
  13.739526940311373,
  14.037199277445273,
  14.208728628673294,
  15.53012550472387,
  15.968462244306963,
  13.875302354622944,
  13.965102288083681,
  13.867669657210797,
  14.01848161574762,
  14.122710076597656,
  14.239434476031516,
  14.655884196808868,
  15.028041698616855,
  15.990194678360082,
  16.31764290964311,
  14.811026856979398,
  15.609685360727966,
  15.194707489852116,
  13.869600547008519,
  13.533548236842545,
  14.765189086033512,
  15.183785573767336,
  13.757923707547482,
  14.75743677773833,
  14.90029921072993,
  13.45910429302221

In [137]:
dictionary_PTS_for_all_change

{1: [10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884167027412,
  10.05884

# Changing whole Training TS with SP and EP for PTS

In [72]:
# for y in range(1,4):


# #     p = 181
#     if(y==1):

#         Sub_Sampling_PTS = Sub_Sampling[:][1:181*y]
#         Pseudo_TS = Sub_Sampling_PTS
#         Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS,y)
#         Run_all_methods(Complete_PTS) 
# #         dictionary_PTS_for_all_change[y] = PTS_Results_List
# #         PTS_Results_List = []
#     else:

#         Sub_Sampling_PTS = Sub_Sampling[:][1+p:181*y]
#         Pseudo_TS = Sub_Sampling_PTS
#         Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS,y)
#         Run_all_methods(Complete_PTS) 
#         dictionary_PTS_for_all_change[y] = PTS_Results_List
#         PTS_Results_List = []

## Functions body for Pseudo TS

In [125]:
def generate_similarity_matrix(Complete_PTS):
    ED_Matrix = distance.cdist(Complete_PTS, Complete_PTS, 'euclidean')
    return ED_Matrix

In [126]:
def generate_graph(similarity_output):
    G = nx.from_numpy_matrix(similarity_output, create_using=nx.Graph)
    layout = nx.spring_layout(G)
    sizes = len(similarity_output)
    return G

In [127]:
def generate_MST(G):
    layout = nx.spring_layout(G)
    T=nx.minimum_spanning_tree(G)
    return T

In [128]:
def generate_Dijkstra(T):
    Dij = nx.dijkstra_path(T,0,181)
    return Dij

In [129]:
def doing_miscellaneous_work(Dijkstra_output):
    a = []
    for x in range(len(Dijkstra_output)):
    
        Index = Dijkstra_output[x]
        a.append(list(Complete_PTS[Index]))
    P_TS = np.array(a)
    return P_TS

In [130]:
def generate_DTW_PTS(Miscellaneous_output):
    path, dist = dtw_path(finalarray_test_data, Miscellaneous_output)
    if(dist>0):
        PTS_Results_List.append(dist)     

In [131]:
def Adding_start_end_point_PTS(Pseudo_TS,y):
    if(y==1):
        SP = [Testing_View[:][0]]
        EP = [Testing_View[:][181]]
        Output_PTS = np.concatenate((SP, Pseudo_TS),axis=0)
        Output_PTS = np.concatenate((Output_PTS, EP),axis=0)
    else:
        p =182
        SP = [Testing_View[:][p*(y-1)]]
        EP = [Testing_View[:][((y*p)-1)]]
        Output_PTS = np.concatenate((SP, Pseudo_TS),axis=0)
        Output_PTS = np.concatenate((Output_PTS, EP),axis=0)
    return Output_PTS

In [132]:
def Run_all_methods(Complete_PTS):
    similarity_output = generate_similarity_matrix(Complete_PTS)
    graph_output = generate_graph(similarity_output)
    MST_output_1 =  generate_MST(graph_output)
    Dijkstra_output = generate_Dijkstra(MST_output_1)
    Miscellaneous_output = doing_miscellaneous_work(Dijkstra_output)
    generate_DTW_PTS(Miscellaneous_output)